compare calibration result and simulation result in the instant loop detector

In [3]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-02'
detector = 'e2w_in'
number = 0
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [4]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [5]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,e2w_in,1.577923e+09,enter,0_e2w_in,13.33,5.00,DEFAULT_VEHTYPE,None
3,e2w_in,1.577923e+09,enter,1_e2w_in,11.66,5.00,DEFAULT_VEHTYPE,4.16
5,e2w_in,1.577923e+09,enter,2_e2w_in,10.41,5.00,DEFAULT_VEHTYPE,3.16
8,e2w_in,1.577923e+09,enter,3_e2w_in,10.41,5.00,DEFAULT_VEHTYPE,1.36
11,e2w_in,1.577923e+09,enter,4_e2w_in,10.41,5.00,DEFAULT_VEHTYPE,1.24
...,...,...,...,...,...,...,...,...
24682,e2w_in,1.578009e+09,enter,9476_e2w_in,10.00,5.00,DEFAULT_VEHTYPE,35.05
24685,e2w_in,1.578010e+09,enter,9477_e2w_in,7.08,5.00,DEFAULT_VEHTYPE,10.28
24688,e2w_in,1.578010e+09,enter,9478_e2w_in,12.50,5.00,DEFAULT_VEHTYPE,43.79
24691,e2w_in,1.578010e+09,enter,9479_e2w_in,10.83,5.00,DEFAULT_VEHTYPE,4.51


In [6]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_e2w_in,1.577923e+09,13.328000,1.60,1577923226,1577923208,max,13.333333,-0.27,-0.005333
1,1_e2w_in,1.577923e+09,11.661995,1.40,1577923230,1577923210,max,11.944444,0.26,-0.282450
2,2_e2w_in,1.577923e+09,10.412498,1.25,1577923233,1577923211,max,8.888889,1.10,1.523609
3,3_e2w_in,1.577923e+09,10.412500,1.25,1577923235,1577923213,max,8.888889,0.95,1.523611
4,4_e2w_in,1.577923e+09,10.829001,1.30,1577923237,1577923214,max,8.611111,0.44,2.217890


In [7]:
print(len(data), len(df_instant_out))

9481 9481


compare time error and speed error in loop detector between simulation result and calibration result

In [8]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_e2w_in,1.577923e+09,13.33,1.577923e+09,13.328000,1577923226,13.333333
1,1_e2w_in,1.577923e+09,11.66,1.577923e+09,11.661995,1577923230,11.944444
2,2_e2w_in,1.577923e+09,10.41,1.577923e+09,10.412498,1577923233,8.888889
3,3_e2w_in,1.577923e+09,10.41,1.577923e+09,10.412500,1577923235,8.888889
4,4_e2w_in,1.577923e+09,10.41,1.577923e+09,10.829001,1577923237,8.611111


In [9]:
#compare_df = compare_df.head(19)

In [10]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.4908237415279923)

In [11]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(0.5995886416868272)

In [12]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(0.6284533800929797)

In [13]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(0.7173487325528248)

In [14]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-4,  # Bin start
        end=11,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [15]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-7.25 10.410000085830688
-5.710000038146973 7.1600000858306885


In [16]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

-6.734444444444445 7.496666666666667
-2.634403025087888 8.231639080794608


In [17]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-7,  # Bin start
        end=7,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

In [18]:
#import os
#os.environ["HORNSGATAN_HOME"]
